# Simulated Annealing on MaxCut Problem

In this notebook, we demonstrate Simulated Annealing on the MaxCut problem.

1. [Introduction](#intro)

<a id='intro'></a>
## 1. Introduction

### Imports

In [ ]:
%pip install copy
%pip install time
%pip install numpy
%pip install random
%pip install random
%pip install networkx